In [1]:
# Importing libraries
from pyedlut import simulation_wrapper as pyedlut
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.stats import norm
import scipy.stats as stats 
import seaborn as sns
from scipy.ndimage import gaussian_filter

## Importing functions
import sys
sys.path.insert(1, '../data')
import figure_2_functions as fun
import simulation_function as sim
import plot_spikes_function as scatter

In [2]:
## Creating plots directory

RESULTS_DIR = "../results/figures_plots"
FIGURE_2_RESULTS_DIR = os.path.join(RESULTS_DIR, 'figure_2')
os.makedirs(FIGURE_2_RESULTS_DIR, exist_ok = True)

In [3]:
## Creating default parameters dictionary
my_dict_default = {"fraction":0, 
            "pattern": 1,
            "sigma": 5, 
            "seed": 0,
            "poisson_seed": 0,
            "seed_noise": 0,
            "noise": 0.0, 
            "mf_goc_w": 0.0,
            "grc_goc_w": 0.0,
            "num_patterns": 12,
            "duration_pattern": 0.25,
            "interval": 0.25,
            "firing_rate": 50.0, 
            "step": 5 
}

# OVERVIEW

In [4]:
## Creating dictionary to initially visualize the spikes
dict_fig_overview = {
    **my_dict_default,
    "fraction": 4,
    "num_patterns": 30, 
    "mf_goc_w": 0.10,
    "grc_goc_w": 0.04,
    "duration_pattern": 0.08, 
    "interval": 0.07
}

In [ ]:
## Retrieving activity
mf_activity = fun.mf_activity_overview(dict_fig_overview)
## Running simulation
oi, ot, _, _ = sim.simulation(dict_fig_overview, mf_activity)
## Plotting spikes
fig = scatter.plot_spikes(oi, ot, dict_fig_overview, 0.0, 10*dict_fig_overview["duration_pattern"])

# FIRST PATTERN (auditory stimulus simulation)

In [6]:
## Setting parameter dictionary for the stimulus without inhibition
dict_p1_without = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 7, 

}

## Setting parameter dictionary for the stimulus with inhibition
dict_p1_with = {
    **my_dict_default,
    "fraction": 4,
    "pattern": 7, 
    "mf_goc_w": 0.20,
    "grc_goc_w": 0.05
}



In [ ]:
## Running simulations of the stimuli presentations
## Retrieving GrCs firing rates matrices
mf_activity = fun.mf_activity_1(dict_p1_without)
_, _, matrix_fr_without, _ = sim.simulation(dict_p1_without, mf_activity)

mf_activity = fun.mf_activity_1(dict_p1_with)
_, _, matrix_fr_with, _ = sim.simulation(dict_p1_with, mf_activity)



## Mean responses of GrCs for an "auditory" stimulus 

In [8]:
## Calculation of mean firing rate before stimulation
mean_with = matrix_fr_with.mean()
mean_without = matrix_fr_without.mean()
## Calculating the mean before the stimulus presentation
mean_before_with = matrix_fr_with[:,:20].mean()
mean_before_without = matrix_fr_without[:,:20].mean()

## Joining in one matrix both with and without inhibition results
matrix_fr = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], 2))

# Dividing by their mean before the stimulus presentation, as we want ΔF/F
matrix_fr[:, :, 0] = matrix_fr_with/mean_before_with
matrix_fr[:, :, 1] = matrix_fr_without/mean_before_without


In [ ]:
## selecting number of GrCs that we want to plot (300 for the heatmap)
num = 300  
np.random.seed(6)
set_index = np.random.choice(3760, num)

fig, axn = plt.subplots(1, 2, figsize=(8, 5.5), sharex=True, sharey=True)

#title_fig = "GRC FIRING RATE DURING ONE PATTERN PRESENTATION"
#fig.suptitle(title_fig, fontsize=16)


cbar_ax = fig.add_axes([.93, .125, .04, .76]) #left, bottom, width, height)
plt.subplots_adjust(wspace=0.2, 
                    hspace=0.4)
plt.rcParams["savefig.facecolor"] = "white"
for i, ax in enumerate(axn.flat):
    
    ## smoothing the matrix
    smoothed_matrix = gaussian_filter(matrix_fr[set_index,:,i], sigma=1.0)
    
    ## plotting heatmap
    sns.heatmap(smoothed_matrix, ax=ax, cmap = 'jet',
                cbar=i == 0,
                vmin=0, vmax=50,
                cbar_ax=None if i else cbar_ax)
    
    ticksx = np.arange(0, matrix_fr.shape[1], matrix_fr.shape[1]/4)
    ticksy = np.arange(0, num, num/6).astype(int)
    
    if i == 0:
        ax.set_title('', fontsize = 16)
    else:
        ax.set_title('', fontsize = 16)

    ax.set_xticks([])
    ax.set_yticks(ticksy)
    ax.set_yticklabels(ticksy, fontsize=16)
    
    for spine in ax.spines.values():
        spine.set_visible(True)
    
    for spine in ax.spines.values():
        spine.set(visible=True, lw=.8, edgecolor="black")
    
    if i == 0:
        ax.set_ylabel('Number of cells', fontsize = 18)
    else:
        ax.set_ylabel('')

    ax.tick_params(axis='x', rotation=0)  
    
    if i == 1:        
        ins2 = ax.inset_axes([0.70,-0.08,0.29,0.01])
        ins2.spines[['left', 'top', 'right']].set_visible(False)
        ins2.set_yticks([])
        ins2.set_xticks([])
        ins2.set_xlabel("1 s", fontsize = 14)

         
for spine in cbar_ax.spines.values():
    spine.set_visible(True)
    
for spine in cbar_ax.spines.values():
    spine.set(visible=True, lw=.8, edgecolor="black")        

cbar_ax.tick_params(labelsize= 16)
cbar_ax.set_ylabel("ΔF/F", fontsize = 20, rotation = 270, labelpad=30)   

fig.savefig(FIGURE_2_RESULTS_DIR + '/heatmap_pattern_1.png', bbox_inches="tight", dpi = 600)


## Repetition of stimulus in different trials

In [ ]:
## Repeating the pattern for 10 trials (updating the seed of the poisson generator for every trial)
times = 10

## Running simulations
mf_activity = fun.mf_activity_1(dict_p1_without)
_, _, matrix_fr_without, _ = sim.simulation(dict_p1_without, mf_activity)

mf_activity = fun.mf_activity_1(dict_p1_with)
_, _, matrix_fr_with, _ = sim.simulation(dict_p1_with, mf_activity)


# Dividing by their mean before the stimulus presentation, as we want ΔF/F
mean_before_with = matrix_fr_with[:,:20].mean()
mean_before_without = matrix_fr_without[:,:20].mean()

## Creating matrices to store average firing rates of trials
matrix1 = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], times))
matrix2 = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], times))
matrix1[:,:,0] = matrix_fr_with/mean_before_with
matrix2[:,:,0] = matrix_fr_without/mean_before_without

for i in range(times-1):
    
    dict_p1_without = {
        **my_dict_default,
        "fraction": 4,
        "pattern": 7, 
        "poisson_seed": i+1
    }

    dict_p1_with = {
        **my_dict_default,
        "fraction": 4,
        "pattern": 7, 
        "mf_goc_w": 0.20,
        "grc_goc_w": 0.05, 
        "poisson_seed": i+1
    }

    mf_activity = fun.mf_activity_1(dict_p1_without)
    _, _, matrix_fr_without, _ = sim.simulation(dict_p1_without, mf_activity)

    mf_activity = fun.mf_activity_1(dict_p1_with)
    _, _, matrix_fr_with, _ = sim.simulation(dict_p1_with, mf_activity)

    mean_before_with = matrix_fr_with[:,:20].mean()
    mean_before_without = matrix_fr_without[:,:20].mean()
    matrix1[:,:,i+1] = matrix_fr_with/mean_before_with
    matrix2[:,:,i+1] = matrix_fr_without/mean_before_without


## Example of firing rates from a granule cell on sequential pattern presentation trials with and without inhibition

In [ ]:
## Selecting one GrC to show its firing rates
cell = 1603

m_1 = matrix1[cell,:,:].T
m_2 = matrix2[cell,:,:].T
m1 = gaussian_filter(m_1, sigma=0.5)
m2 = gaussian_filter(m_2, sigma=0.5)
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (8, 4))
x = np.arange(0, m1.shape[1], 1)
for i in range(times):
    ax1.plot(x, m1[i,:] - i*28.0, c = 'black')
    ax2.plot(x, m2[i,:] - i*28.0, c = 'red')

ax1.set_ylim([m2[-1,:].min() - (times-1)*32.0, m2[0,:].max() + 40])
ax2.set_ylim([m2[-1,:].min() - (times-1)*32.0, m2[0,:].max() + 40])
    
ax1.axis("off")
ax2.axis("off")

ax1.set_title("Control", color = 'black', fontsize = 14)
ax2.set_title("DART", color = 'red', fontsize = 14)

ins2 = ax2.inset_axes([0.68,-0.08,0.29,0.12])
ins2.spines[['left', 'top']].set_visible(False)
ins2.set_yticks([])
ins2.set_xticks([])
ins2.set_xlabel("1 s", fontsize = 14)

ax1.arrow(-3.0,5 , 0, -255, head_width=4, color = 'black')

ax1.set_ylabel('Trials', fontsize = 14)

# arrows 
ax1.arrow(17, 80, 0, -65, head_width=4, color = 'black')
ax2.arrow(17, 80, 0, -65, head_width=4, color = 'black')
ax1.text(16,80, "---------------------------|", fontsize = 14)
ax2.text(16,80, "----------------------------|", fontsize = 14)

fig.text(0.75,0.1, "100% ΔF/F", fontsize = 14)

fig.text(0.37,0.1, "Cell " + str(cell), fontsize = 14)
fig.text(0.12,0.40, "Trials", fontsize = 16, rotation = 90)
plt.subplots_adjust(wspace=0.1)

fig.savefig(FIGURE_2_RESULTS_DIR + '/trials_cell_' + str(cell) + '_1.png', bbox_inches="tight", dpi = 200)

## Mean time course of responses during pattern presentation with and without inhibition

In [ ]:
## Calculating the average GrC firing rate  for inhibition and no inhibition
smoothed_matrix1 = gaussian_filter(matrix1.mean(axis = 2), sigma=1.5)
smoothed_matrix2 = gaussian_filter(matrix2.mean(axis = 2), sigma=1.5)
response_with = sum(smoothed_matrix1)/matrix_fr_with.shape[0]
response_without = sum(smoothed_matrix2)/matrix_fr_with.shape[0]

plt.rcParams["figure.figsize"] = (2.8,4)
fig, ax = plt.subplots()

time = np.arange(0, matrix_fr_with.shape[1], 1)
ax.scatter(time, response_without, label = 'DART', s =20,  facecolors='none', edgecolors='r')
ax.plot(time, response_without, c = 'red')
ax.scatter(time, response_with, label = 'control', s = 20, facecolors='none', edgecolors='black')
ax.plot(time, response_with, c = 'black')

ax.spines[['right', 'top']].set_visible(False)
ax.set_xticks([0, 20, 40, 60])
ax.set_xticklabels([-1, 0, 1, 2])
plt.tick_params(axis ='both', labelsize = 15)
ax.set_ylabel('Response (ΔF/F)', fontsize = 15)
ax.set_xlabel('Time (s)', fontsize = 15)
leg = ax.legend(fontsize = 12)
colors = ['red', 'black']
for color,text in zip(colors,leg.get_texts()):
    text.set_color(color)
    
fig.savefig(FIGURE_2_RESULTS_DIR + '/avg_firing_rate_1.png', bbox_inches="tight", dpi = 200)

## Mean response amplitude for individual cells (firing rate)

In [13]:
## Calculating the mean response amplitude (firing rate after stimulus substracted by firing rate before stimulus)
## for each cell 
mean1 = smoothed_matrix1.mean()
before = smoothed_matrix1[:, :20]
after = smoothed_matrix1[:, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)

## Mean response amplitude
mean_amp_with = (after - before)

In [14]:
mean2 = smoothed_matrix2.mean()
before = smoothed_matrix2[:, :20]
after = smoothed_matrix2[:, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)

## Mean response amplitude
mean_amp_without = (after - before)

In [15]:
## Creating the dataframes

df_with = pd.DataFrame(mean_amp_with)
df_without = pd.DataFrame(mean_amp_without)
df_with['Type'] = ['Control']*mean_amp_with.shape[0]
df_with = df_with.rename(columns={0: 'Mean amplitude'})
df_without['Type'] = ['DART']*mean_amp_without.shape[0]
df_without = df_without.rename(columns={0: 'Mean amplitude'})

## Selecting only a small subset of GrCs
x = np.random.choice(mean_amp_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)

# Saving here all the samples results
df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:
# Performing the paired sample t-test 
## Notice that P-values results were calculated and then they were manually written in every plot
## Therefore they should be edited for each combination of parameters computed
stats.ttest_rel(mean_amp_with, mean_amp_without) 

In [ ]:
## Plotting swarmplots with all GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Mean amplitude", data=df_probs, size = 3, palette = my_pal, alpha = 1.0, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 15)
ax.set(xlabel=None)

df_mean = df_probs2.groupby('Type', sort=False)['Mean amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Mean amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 38, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/mean_response_amplitude_1.png', bbox_inches="tight", dpi = 200)

In [ ]:
## Plotting violinplots with all GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y="Mean amplitude", data=df_probs2, palette = my_pal, alpha = 1.0)
ax.spines[['right', 'top']].set_visible(False)

ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 15)
ax.set(xlabel=None)

df_mean = df_probs2.groupby('Type', sort=False)['Mean amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Mean amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--') 
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 38, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/mean_response_amplitude_violin_1.png', bbox_inches="tight", dpi = 200)

In [ ]:
## Checking mean
df_mean

In [ ]:
# Checking standard deviation
df_sem = df_probs2.groupby('Type', sort=False)['Mean amplitude'].sem()
df_sem

## Mean spike probability for each cell 

In [21]:
## Calculating the spike probability for each GrC 
prob_with = np.sum(smoothed_matrix1, axis = 1)/np.sum(smoothed_matrix1, axis = 1).max()
prob_without = np.sum(smoothed_matrix2, axis = 1)/np.sum(smoothed_matrix2, axis = 1).max()
df_with = pd.DataFrame(prob_with)
df_without = pd.DataFrame(prob_without)
df_with['Type'] = ['Control']*prob_with.shape[0]
df_with = df_with.rename(columns={0: 'Single Trial Probability'})
df_without['Type'] = ['DART']*prob_without.shape[0]
df_without = df_without.rename(columns={0: 'Single Trial Probability'})

## Selecting only a small subset of GrCs
x = np.random.choice(prob_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)

# Saving here all the samples results
df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:
# Performing the paired sample t-test 
stats.ttest_rel(df_with['Single Trial Probability'], df_without['Single Trial Probability']) 

In [ ]:
## Plotting swarmplots with the subset of GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Single Trial Probability", data=df_probs, size = 3, palette = my_pal, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Pr.', fontsize = 15)
ax.set_ylim([0.0, 1.0])
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Probability'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Probability'].items()]
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 0.98, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/spike_probability_1.png', bbox_inches="tight", dpi = 200)

In [ ]:
## Plotting violinplots with all GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y="Single Trial Probability", data=df_probs2, palette = my_pal, alpha = 1.0, cut = 0)
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Pr.', fontsize = 15)
ax.set_ylim([0.0, 1.0])
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Probability'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Probability'].items()]
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 0.98, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/spike_probability_violin_1.png', bbox_inches="tight", dpi = 200)


In [ ]:
df_mean

In [ ]:
df_sem = df_probs2.groupby('Type', sort=False)["Single Trial Probability"].sem()
df_sem

## Single trial amplitude (firing rate) 

In [27]:
array1 = (matrix1[:, 20:, 0].mean(axis = 1))
array2 = (matrix2[:, 20:, 0].mean(axis = 1))

## Considering only the significant GrCs for the inhibition case
significant_1 = np.where(array1 > 0)[0]

In [ ]:
significant_1.shape

In [29]:
mean1 = smoothed_matrix1.mean()
before = smoothed_matrix1[significant_1, :20]
after = smoothed_matrix1[significant_1, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)

## Mean response amplitude
mean_amp_with = (after - before)

In [30]:
mean2 = smoothed_matrix2.mean()
before = smoothed_matrix2[significant_1, :20]
after = smoothed_matrix2[significant_1, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)

## Mean response amplitude
mean_amp_without = (after - before)

In [31]:
## Creating the dataframes

df_with = pd.DataFrame(mean_amp_with)
df_without = pd.DataFrame(mean_amp_without)
df_with['Type'] = ['Control']*mean_amp_with.shape[0]
df_with = df_with.rename(columns={0: 'Single Trial Amplitude'})
df_without['Type'] = ['DART']*mean_amp_without.shape[0]
df_without = df_without.rename(columns={0: 'Single Trial Amplitude'})

## Selecting only a small subset of GrCs
x = np.random.choice(mean_amp_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)

# Saving here all the samples results
df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:
# Performing the paired sample t-test 
stats.ttest_rel(df_with['Single Trial Amplitude'], df_without['Single Trial Amplitude']) 

In [ ]:
## Plotting swarmplots with the subset of GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Single Trial Amplitude", data=df_probs, size = 3, palette = my_pal, alpha = 1.0, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Amp. (ΔF/F)', fontsize = 15)

df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Amplitude'].items()]
 
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 39, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/single_trial_amplitude_1.png', bbox_inches="tight", dpi = 200)

In [ ]:
## Plotting violinplots with all GrCs 

fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y="Single Trial Amplitude", data=df_probs2,palette = my_pal, alpha = 1.0, cut = 0)
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Amp. (ΔF/F)', fontsize = 15)
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Amplitude'].items()]

plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 39, '****', fontsize = 15)

fig.savefig(FIGURE_2_RESULTS_DIR + '/single_trial_amplitude_violin_1.png', bbox_inches="tight", dpi = 200)


In [ ]:
df_mean

In [ ]:
df_sem = df_probs2.groupby('Type', sort=False)["Single Trial Amplitude"].sem()
df_sem

# SECOND PATTERN (somatosensory stimulus simulation)

In [37]:
## Creating dictionary to initially visualize the spikes

dict_fig_overview = {
    **my_dict_default,
    "fraction": 4,
    "sigma": 40,
    "firing_rate": 100.0,
    "num_patterns": 30, 
    "mf_goc_w": 0.10,
    "grc_goc_w": 0.04,
    "duration_pattern": 0.08, 
    "interval": 0.07
}

In [ ]:
## Rretrieving activity
mf_activity = fun.mf_activity_overview(dict_fig_overview)
## Running simulation
oi, ot, _, matrix = sim.simulation(dict_fig_overview, mf_activity)
## Plotting spikes
fig = scatter.plot_spikes(oi, ot, dict_fig_overview, 0.0, 10*dict_fig_overview["duration_pattern"])

In [39]:
## Setting parameter dictionary for the stimulus without inhibition
dict_p2_without = {
    **my_dict_default,
    "fraction": 3,
    "sigma": 40,
    "pattern": 11, 
    "firing_rate": 80.0,
}

## Setting parameter dictionary for the stimulus with inhibition
dict_p2_with = {
    **my_dict_default,
    "fraction": 3,
    "sigma": 40,
    "pattern": 11, 
    "mf_goc_w": 0.20,
    "grc_goc_w": 0.02,
    "firing_rate": 80.0,
}


In [ ]:
## Running the stimulus without inhibition
mf_activity = fun.mf_activity_2(dict_p2_without)
_, _, matrix_fr_without, _ = sim.simulation(dict_p2_without, mf_activity)

## Running the stimulus with inhibition
mf_activity = fun.mf_activity_2(dict_p2_with)
_, _, matrix_fr_with, _ = sim.simulation(dict_p2_with, mf_activity)

## Calculating the mean before the stimulus presentation
mean_before_with = matrix_fr_with[:,:20].mean()
mean_before_without = matrix_fr_without[:,:20].mean()

## Joining in one matrix both with and without inhibition results
matrix_fr = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], 2))

# Dividing by their mean before the stimulus presentation, as we want ΔF/F
matrix_fr[:, :, 0] = matrix_fr_with/mean_before_with
matrix_fr[:, :, 1] = matrix_fr_without/mean_before_without


In [ ]:
## selecting number of GrCs that we want to plot (300 for the heatmap)
np.random.seed(4)
set_index = np.random.choice(3760, 300)
num = 300
fig, axn = plt.subplots(1, 2, figsize=(8, 5.5), sharex=True, sharey=True)

cbar_ax = fig.add_axes([.93, .125, .04, .76]) #left, bottom, width, height)

plt.subplots_adjust(wspace=0.2, 
                    hspace=0.4)

plt.xticks(fontsize=20)

for i, ax in enumerate(axn.flat):

    smoothed_matrix = gaussian_filter(matrix_fr[set_index,:,i], sigma=1.0)
    sns.heatmap(smoothed_matrix, ax=ax, cmap = 'jet',
                cbar=i == 0,
                vmin=0, vmax=110,
                cbar_ax=None if i else cbar_ax)

    ticksx = np.arange(0, matrix_fr.shape[1], matrix_fr.shape[1]/4)
    ticksy = np.arange(0, num, num/6).astype(int)

    ax.set_xticks([])
    ax.set_yticks(ticksy)
    ax.set_yticklabels(ticksy, fontsize=16)

    for spine in ax.spines.values():
        spine.set_visible(True)

    for spine in ax.spines.values():
        spine.set(visible=True, lw=.8, edgecolor="black")

    if i == 0:
        ax.set_ylabel('Number of cells', fontsize = 16)
    else:
        ax.set_ylabel('')

    ax.tick_params(axis='x', rotation=0)  

    if i == 1:        
        ins2 = ax.inset_axes([0.70,-0.08,0.29,0.01])
        ins2.spines[['left', 'top', 'right']].set_visible(False)
        ins2.set_yticks([])
        ins2.set_xticks([])
        ins2.set_xlabel("1 s", fontsize = 16)


for spine in cbar_ax.spines.values():
    spine.set_visible(True)

for spine in cbar_ax.spines.values():
    spine.set(visible=True, lw=.8, edgecolor="black")        

cbar_ax.tick_params(labelsize= 16)
cbar_ax.set_ylabel("ΔF/F", fontsize = 20, rotation = 270, labelpad=30)      
plt.show()
fig.savefig(FIGURE_2_RESULTS_DIR + '/heatmap_pattern_2.png', bbox_inches="tight", dpi = 600)

## Repetition of stimulus in different trials

In [ ]:
## Repeating the pattern for 10 trials (updating the seed of the poisson generator for every trial)
times = 10

## Running simulations
mf_activity = fun.mf_activity_2(dict_p2_without)
_, _, matrix_fr_without, _ = sim.simulation(dict_p2_without, mf_activity)

mf_activity = fun.mf_activity_2(dict_p2_with)
_, _, matrix_fr_with, _ = sim.simulation(dict_p2_with, mf_activity)

## Calculating means before pattern presenation
mean_before_with = matrix_fr_with[:,:20].mean()
mean_before_without = matrix_fr_without[:,:20].mean()

## Creating matrices to store average firing rates of trials
matrix1 = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], times))
matrix2 = np.zeros((matrix_fr_with.shape[0], matrix_fr_with.shape[1], times))
matrix1[:,:,0] = matrix_fr_with/mean_before_with
matrix2[:,:,0] = matrix_fr_without/mean_before_without

for i in range(times-1):
    
    dict_p2_without = {
        **my_dict_default,
        "fraction": 3,
        "sigma": 40,
        "pattern": 11, 
        "poisson_seed": i+1,
        "firing_rate": 80.0,
    }

    dict_p2_with = {
        **my_dict_default,
        "fraction": 3,
        "sigma": 40,
        "pattern": 11, 
        "mf_goc_w": 0.20,
        "grc_goc_w": 0.02,
        "poisson_seed": i+1,
        "firing_rate": 80.0,
    }


    mf_activity = fun.mf_activity_2(dict_p2_without)
    _, _, matrix_fr_without, _ = sim.simulation(dict_p2_without, mf_activity)

    mf_activity = fun.mf_activity_2(dict_p2_with)
    _, _, matrix_fr_with, _ = sim.simulation(dict_p2_with, mf_activity)

    mean_before_with = matrix_fr_with[:,:20].mean()
    mean_before_without = matrix_fr_without[:,:20].mean()
    matrix1[:,:,i+1] = matrix_fr_with/mean_before_with
    matrix2[:,:,i+1] = matrix_fr_without/mean_before_without

## Example of firing rates from a granule cell on sequential pattern presentation trials with and without inhibition

In [ ]:
## Selecting one GrC to show its firing rates
cell = 694
m_1 = matrix1[cell,:,:].T
m_2 = matrix2[cell,:,:].T
m1 = gaussian_filter(m_1, sigma=0.5)
m2 = gaussian_filter(m_2, sigma=0.5)
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (8, 4))
x = np.arange(0, m1.shape[1], 1)
for i in range(times):
    ax1.plot(x, m1[i,:] - i*28.0, c = 'black')
    ax2.plot(x, m2[i,:] - i*28.0, c = 'red')

ax1.set_ylim([m2[-1,:].min() - (times-1)*32.0, m2[0,:].max() + 40])
ax2.set_ylim([m2[-1,:].min() - (times-1)*32.0, m2[0,:].max() + 40])
    
ax1.axis("off")
ax2.axis("off")

ax1.set_title("Control", color = 'black', fontsize = 14)
ax2.set_title("DART", color = 'red', fontsize = 14)

ins2 = ax2.inset_axes([0.68,-0.08,0.29,0.12])
ins2.spines[['left', 'top']].set_visible(False)
ins2.set_yticks([])
ins2.set_xticks([])
ins2.set_xlabel("1 s", fontsize = 14)

ax1.arrow(-3.,5 , 0, -255, head_width=4, color = 'black')
ax1.set_ylabel('Trials', fontsize = 14)

ax1.arrow(16, 95, 0, -80, head_width=4, color = 'black')
ax2.arrow(16, 95, 0, -80, head_width=4, color = 'black')
ax1.text(15, 95, "----------|", fontsize = 14)
ax2.text(15, 95, "----------|", fontsize = 14)


fig.text(0.75,0.1, "100% ΔF/F", fontsize = 14)
fig.text(0.37,0.1, "Cell " + str(cell), fontsize = 14)
fig.text(0.12,0.40, "Trials", fontsize = 16, rotation = 90)
plt.subplots_adjust(wspace=0.1)
fig.savefig(FIGURE_2_RESULTS_DIR + '/trials_cell_' + str(cell) + '_2.png', bbox_inches="tight", dpi = 200)


## Mean time course of responses during pattern presentation with and without inhibition

In [ ]:
## Calculating the average firing rate of the GrC for inhibition and no inhibition
smoothed_matrix1 = gaussian_filter(matrix1.mean(axis = 2), sigma=1.5)
smoothed_matrix2 = gaussian_filter(matrix2.mean(axis = 2), sigma=1.5)
response_with = sum(smoothed_matrix1)/matrix_fr_with.shape[0]
response_without = sum(smoothed_matrix2)/matrix_fr_with.shape[0]

plt.rcParams["figure.figsize"] = (2.8,4)
fig, ax = plt.subplots()


time = np.arange(0, matrix_fr_with.shape[1], 1)
ax.scatter(time, response_without, label = 'DART', s =20,  facecolors='none', edgecolors='r')
ax.plot(time, response_without, c = 'red')
ax.scatter(time, response_with, label = 'control', s = 20, facecolors='none', edgecolors='black')
ax.plot(time, response_with, c = 'black')

ax.spines[['right', 'top']].set_visible(False)
ax.set_xticks(np.arange(0, matrix_fr_with.shape[1], matrix_fr_with.shape[1]/4))
ax.set_xticks([0, 20, 40, 60])
ax.set_xticklabels([-1, 0, 1, 2])
ax.set_ylabel('Response (ΔF/F)', fontsize = 15)
ax.set_xlabel('Time (s)', fontsize = 15)
leg = ax.legend(fontsize = 10)
colors = ['red', 'black']
for color,text in zip(colors,leg.get_texts()):
    text.set_color(color)
    
plt.tick_params(axis ='both', labelsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/avg_firing_rate_2.png', bbox_inches="tight", dpi = 200)


## Mean response amplitude for individual cells (firing rate)

In [45]:
mean1 = smoothed_matrix1.mean()
before = smoothed_matrix1[:, :20]
after = smoothed_matrix1[:, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
mean_amp_with = (after - before)

In [46]:
mean2 = smoothed_matrix2.mean()
before = smoothed_matrix2[:, :20]
after = smoothed_matrix2[:, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
mean_amp_without = (after - before)

In [47]:
df_with = pd.DataFrame(mean_amp_with)
df_without = pd.DataFrame(mean_amp_without)
df_with['Type'] = ['Control']*mean_amp_with.shape[0]
df_with = df_with.rename(columns={0: 'Mean amplitude'})
df_without['Type'] = ['DART']*mean_amp_without.shape[0]
df_without = df_without.rename(columns={0: 'Mean amplitude'})
np.random.seed(0)

x = np.random.choice(mean_amp_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)

df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:

# Performing the paired sample t-test 
stats.ttest_rel(df_with['Mean amplitude'], df_without['Mean amplitude']) 

In [ ]:
fig, ax = plt.subplots() 
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Mean amplitude", data=df_probs, size = 3, palette = my_pal, alpha = 1.0, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 15)

ax.set(xlabel=None)
df_mean = df_probs2.groupby('Type', sort=False)['Mean amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Mean amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--')
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 40, '****', fontsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/mean_response_amplitude_2.png', bbox_inches="tight", dpi = 200)


In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y="Mean amplitude", data=df_probs2, palette = my_pal, alpha = 1.0)
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Av. Amp. (ΔF/F)', fontsize = 15)

ax.set(xlabel=None)
df_mean = df_probs2.groupby('Type', sort=False)['Mean amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Mean amplitude'].items()]
ax.axhline(y = 0.0, color = 'black', linestyle = '--')
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 65, '****', fontsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/mean_response_amplitude_violin_2.png', bbox_inches="tight", dpi = 200)



In [ ]:
df_mean

In [ ]:
df_sem = df_probs2.groupby('Type', sort=False)['Mean amplitude'].sem()
df_sem

## Mean spike probability for each cell 

In [53]:
prob_with = np.sum(smoothed_matrix1, axis = 1)/np.sum(smoothed_matrix1, axis = 1).max()
prob_without = np.sum(smoothed_matrix2, axis = 1)/np.sum(smoothed_matrix2, axis = 1).max()
df_with = pd.DataFrame(prob_with)
df_without = pd.DataFrame(prob_without)
df_with['Type'] = ['Control']*prob_with.shape[0]
df_with = df_with.rename(columns={0: 'Single Trial Probability'})
df_without['Type'] = ['DART']*prob_without.shape[0]
df_without = df_without.rename(columns={0: 'Single Trial Probability'})
x = np.random.choice(prob_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)
df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:
# Performing the paired sample t-test 
stats.ttest_rel(df_with['Single Trial Probability'], df_without['Single Trial Probability']) 

In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Single Trial Probability", data=df_probs, size = 3, palette = my_pal, alpha = 1.0, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Pr.', fontsize = 15)
ax.set_ylim([0.0, 1.0])
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Probability'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Probability'].items()]
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 1.0, '****', fontsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/spike_probability_2.png', bbox_inches="tight", dpi = 200)



In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y='Single Trial Probability', data=df_probs2, palette = my_pal, alpha = 1.0, cut = 0)
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Pr.', fontsize = 15)
ax.set_ylim([0.0, 1.0])
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Probability'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Probability'].items()]
plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3, 1.0, '****', fontsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/spike_probability_violin_2.png', bbox_inches="tight", dpi = 200)



In [ ]:
df_mean

In [ ]:
df_sem = df_probs2.groupby('Type', sort=False)["Single Trial Probability"].sem()
df_sem

## Single trial amplitude (firing rate) 

In [59]:
array1 = (matrix1[:, 20:30, 0].mean(axis = 1))
array2 = (matrix2[:, 20:30, 0].mean(axis = 1))
significant_1 = np.where(array1 > 0)[0]
significant_2 = np.where(array2 > 0)[0]

In [ ]:
significant_1.shape

In [61]:
mean1 = smoothed_matrix1.mean()
before = smoothed_matrix1[significant_1, :20]
after = smoothed_matrix1[significant_1, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
mean_amp_with = (after - before)

In [62]:
mean2 = smoothed_matrix2.mean()
before = smoothed_matrix2[significant_1, :20]
after = smoothed_matrix2[significant_1, 20:]
before = before.mean(axis = 1)
after = after.mean(axis = 1)
mean_amp_without = (after - before)

In [63]:
df_with = pd.DataFrame(mean_amp_with)
df_without = pd.DataFrame(mean_amp_without)
df_with['Type'] = ['Control']*mean_amp_with.shape[0]
df_with = df_with.rename(columns={0: 'Single Trial Amplitude'})
df_without['Type'] = ['DART']*mean_amp_without.shape[0]
df_without = df_without.rename(columns={0: 'Single Trial Amplitude'})
np.random.seed(0)

x = np.random.choice(mean_amp_with.shape[0], 400)
df_probs = pd.concat([df_with.iloc[x], df_without.iloc[x]], axis=0)

df_probs2 = pd.concat([df_with, df_without], axis=0)

In [ ]:
# Performing the paired sample t-test 
stats.ttest_rel(df_with['Single Trial Amplitude'], df_without['Single Trial Amplitude']) 

In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.swarmplot(x="Type", y="Single Trial Amplitude", data=df_probs, size = 3, palette = my_pal, alpha = 1.0, edgecolors = 'black')
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Amp. (ΔF/F)', fontsize = 15)
ax.set_ylim(bottom=0.0, top=None)
df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Amplitude'].items()]

plt.tick_params(axis ='both', labelsize = 15)
plt.text(0.3,59, '****', fontsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/single_trial_amplitude_2.png', bbox_inches="tight", dpi = 200)



In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (2.8,4)
my_pal = {"Control": "black", "DART": "red"}

ax = sns.violinplot(x="Type", y="Single Trial Amplitude", data=df_probs2, palette = my_pal, alpha = 1.0, cut = 0)
ax.spines[['right', 'top']].set_visible(False)
ax.set(xlabel=None)
ax.set_ylabel('Single Trial Amp. (ΔF/F)', fontsize = 15)

df_mean = df_probs2.groupby('Type', sort=False)['Single Trial Amplitude'].mean()
_ = [ax.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=2) for i, y in df_mean.reset_index()['Single Trial Amplitude'].items()]

plt.text(0.3, 65, '****', fontsize = 15)
plt.tick_params(axis ='both', labelsize = 15)
fig.savefig(FIGURE_2_RESULTS_DIR + '/single_trial_amplitude_violin_2.png', bbox_inches="tight", dpi = 200)


In [ ]:
df_mean

In [ ]:
df_sem = df_probs2.groupby('Type', sort=False)['Single Trial Amplitude'].sem()
df_sem